In [16]:
import sys
sys.path.append("../")
from monitoring.data_collector import Online_Collector, Batch_Collector
from monitoring.data_drift import Drift_Analysis
import pandas as pd
import time
from azureml.core import Workspace
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
wsname = "ws01ent"
rg = "azureml"
subid = "0e9bace8-7a81-4922-83b5-d995ff706507"
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id=tenant_id)

# ws = Workspace(workspace_name=wsname, resource_group=rg, subscription_id=subid, auth=interactive_auth)
ws = Workspace.from_config()

kv=ws.get_default_keyvault()


Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.


#### Prerequisite
1. Create a service principal and secret (SP)
2. Provision an ADX cluster and create a DB
3. Add the SP to be contributor of the cluster
4. pip install following packages: pip install --upgrade azure-mgmt-eventhub azure-eventhub azure-mgmt-kusto azure-kusto-ingest azure-kusto-data azure-identity azure-common


#### Provisioning resource


The following code provision target table at ADX, eventhub and ingestion procedure to ingest data automatically once data is available at eventhub

In [17]:
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
#Application ID
client_id = "af883abf-89dd-4889-bdb3-1ee84f68465e"
#Client Secret, set it at your WS' keyvault with key name same as your client_id
client_secret = kv.get_secret(client_id)
subscription_id = "0e9bace8-7a81-4922-83b5-d995ff706507"

cluster_uri = "https://adx02.westus2.kusto.windows.net" #URL of the ADX Cluster

db_name = "db01"
table_name = "isd_weather_test" #new dataset
kv=ws.get_default_keyvault()


sample_pd_data = pd.read_parquet("data/test_data.parquet")
sample_pd_data['timestamp'] = sample_pd_data['datetime']
sample_pd_data.drop(['datetime'], inplace=True, axis=1)
online_collector = Online_Collector(tenant_id, client_id,client_secret,cluster_uri,db_name,table_name, sample_pd_data)

.create table isd_weather_test (usaf: string, wban: string, latitude: real, longitude: real, elevation: real, windAngle: real, windSpeed: real, temperature: real, seaLvlPressure: real, cloudCoverage: string, presentWeatherIndicator: real, pastWeatherIndicator: real, precipTime: real, precipDepth: real, snowDepth: real, stationName: string, countryOrRegion: string, p_k: string, year: int, day: int, version: real, timestamp: datetime)


In [18]:
# time.sleep(120) #It takes about 2 minutes for stream_collect to start working 
# online_collector.stream_collect(sample_pd_data)
# online_collector.batch_collect(sample_pd_data)

In [19]:
analysis = Drift_Analysis(tenant_id, client_id, client_secret, cluster_uri,db_name)
df = analysis.query(f"""
{table_name}| take(10)
""")
df

,usaf,wban,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,cloudCoverage,...,precipTime,precipDepth,snowDepth,stationName,countryOrRegion,p_k,year,day,version,timestamp
0,999999,53878,35.419,-82.557,641.0,<NA>,<NA>,4.4,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,23,1.0,2008-01-23 00:15:00+00:00
1,999999,53878,35.419,-82.557,641.0,<NA>,<NA>,-1.6,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,21,1.0,2008-01-21 22:55:00+00:00
2,999999,53877,35.495,-82.614,656.0,<NA>,<NA>,-2.3,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 8 SSW,US,999999-53877,2008,22,1.0,2008-01-22 04:55:00+00:00
3,999999,53877,35.495,-82.614,656.0,<NA>,<NA>,1.3,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 8 SSW,US,999999-53877,2008,19,1.0,2008-01-19 09:50:00+00:00
4,999999,53877,35.495,-82.614,656.0,<NA>,<NA>,-2.2,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 8 SSW,US,999999-53877,2008,22,1.0,2008-01-22 03:50:00+00:00
5,999999,53877,35.495,-82.614,656.0,<NA>,<NA>,-0.9,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 8 SSW,US,999999-53877,2008,21,1.0,2008-01-21 22:30:00+00:00
6,999999,53878,35.419,-82.557,641.0,<NA>,0.3,1.0,<NA>,,...,1.0,0.0,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,6,1.0,2008-01-06 13:00:00+00:00
7,999999,53878,35.419,-82.557,641.0,<NA>,<NA>,-9.5,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,25,1.0,2008-01-25 06:35:00+00:00
8,999999,53878,35.419,-82.557,641.0,<NA>,<NA>,-4.2,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,16,1.0,2008-01-16 06:20:00+00:00
9,999999,53878,35.419,-82.557,641.0,<NA>,<NA>,4.0,<NA>,,...,<NA>,<NA>,<NA>,ASHEVILLE 13 S,US,999999-53878,2008,28,1.0,2008-01-28 14:50:00+00:00


In [20]:
import matplotlib.pyplot as plt

In [21]:
df = analysis.query(f"""
{table_name}| summarize avg(temperature) by bin(timestamp, 7d) | sort by timestamp asc
""")
df

,timestamp,avg_temperature
0,2007-12-31 00:00:00+00:00,1.0
1,2008-01-14 00:00:00+00:00,-1.45
2,2008-01-21 00:00:00+00:00,-2.016667
3,2008-01-28 00:00:00+00:00,4.0


In [22]:
df = analysis.query(f"""
{table_name}| extend day = floor(timestamp % 7d, 1d) |
summarize avg(temperature) by day
""")
df

,day,avg_temperature
0,2 days,0.1
1,0 days,0.5
2,1 days,-2.25
3,5 days,1.3
4,6 days,1.0
5,4 days,-9.5


In [23]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import dates as mpl_dates
import datetime

In [24]:
output = widgets.Output()
 
def get_values():
    df = analysis.query(f"""
    {text_tblname.value}| take 10
    """)
    return list(df.columns)

def get_type():
    df = analysis.query(f"""
    {text_tblname.value}| take 10
    """)
    num_cols = list(df.select_dtypes('number').columns)
    cat_cols = list(df.select_dtypes(include=["object"]).columns)
    if column_selector.value in num_cols:
        return ["avg", "min", "max"]
    elif column_selector.value in cat_cols:
        return ["euclidian_distance", "unique_values"]

def get_timestamp():
    df = analysis.query(f"""
    {text_tblname.value}| getschema | where ColumnType == "datetime"
    """)
    return list(df["ColumnName"].values)
    
def get_dates():
    df = analysis.query(f"""
    {text_tblname.value}| summarize min(timestamp), max(timestamp)
    """)

    min_max = df.values.tolist()[0]

    w_options = pd.date_range(pd.offsets.MonthBegin().rollback(min_max[0]), pd.offsets.MonthBegin().rollback(min_max[1]),
              freq="MS", inclusive="both").strftime("%b/%Y").tolist()
    return w_options

# create some control elements
text_tblname = widgets.Text(value="isd_weather_test", description="table name", continuous_update=False)
text_tblnamecomp = widgets.Text(value="", description="table name", continuous_update=False)
clusteruri_name = widgets.Text(value="https://adx02.westus2.kusto.windows.net", description="cluster URI", continuous_update=False)
connect_button = widgets.Button(description="Connect")

database_name = widgets.Text(value="db01", description="Database Name", continuous_update=False)
column_selector = widgets.Dropdown(value=get_values()[0], options=get_values(), description="column name")
datecol_selector = widgets.Dropdown(value=get_timestamp()[0], options=get_timestamp(), description="Date Column")
metric_selector = widgets.Dropdown(value=get_type()[0], options=get_type(), description="metric")
freq_selector = widgets.Dropdown(value="daily", options=["daily", "weekly", "monthly"], description="frequency")
date_slider = widgets.SelectionRangeSlider(
    options=get_dates(),
    index=(0,len(get_dates())-1),
    disabled=False,
    description="date range")
test_button = widgets.Button(description="Plot")

startdate_selector = widgets.DatePicker(description="Start Date")
enddate_selector = widgets.DatePicker(description="End Date")

mdl_name = widgets.Text(value="model_name", description="model name", continuous_update=False)

# callback functions
def update_tbl(change):
    df = analysis.query(f"""
    {text_tblname.value}| take 10
    """)
    column_selector.options= list(df.columns)

def update_col(change):
    new_cols = get_type()
    metric_selector.options = new_cols

def get_data():
    freq_dict = {"daily": "1d",
                "weekly": "7d"}
    metric = metric_selector.value
    col = column_selector.value
    start_date = datetime.datetime.strptime(date_slider.value[0], "%b/%Y").strftime("%Y-%m-%d")
    end_date = datetime.datetime.strptime(date_slider.value[1], "%b/%Y").strftime("%Y-%m-%d")
    date_col = datecol_selector.value
    freq = freq_dict[freq_selector.value]
    if metric in ["avg", "min", "max"]:
        df = analysis.query(f"""
        {table_name}
        | summarize {metric}({col}) by bin({date_col}, {freq}) | sort by {date_col} asc
        """)
        # | where timestamp >= datetime({start_date}) and timestamp <= datetime({end_date})
        return df
    # elif metric == "euclidian_distance":
    #     """logic"""
    elif metric == "unique_values":
        df = analysis.query(f"""
        {table_name}
        | summarize dcount({col}) by bin({date_col}, {freq}) | sort by {date_col} asc
        """)
        return df
        

    else:
        return None
def run_button(b):
    output.clear_output(wait=True)
    plt.close('all')
    datafr = get_data()

    x_data = list(datafr.timestamp.values)
    y_data = list(datafr.iloc[:,-1].values)
    with output:
        fig, ax = plt.subplots(1, 1, figsize=(6, 4)) # constrained_layout=True,
        
        # move the toolbar to the bottom
        fig.canvas.toolbar_position = 'bottom'
        # ax.grid(True)    
        line, = ax.plot(x_data, y_data, linestyle='solid')
        ax.set_xlim(left=x_data[0], right=x_data[-1])
        fig.autofmt_xdate()
        date_format = mpl_dates.DateFormatter('%m-%d-%Y')
        ax.xaxis.set_major_formatter(date_format)
        ax.set_ylim(bottom=0, top=max(y_data)*1.1)
        ax.set_title(f"{metric_selector.value} {freq_selector.value} {column_selector.value}")
        fig.suptitle("")
        # fig.canvas.draw()
        fig.show()


# connect callbacks and traits
text_tblname.observe(update_tbl, 'value')
column_selector.observe(update_col, 'value')
test_button.on_click(run_button)

controls1 = widgets.VBox([clusteruri_name, database_name, connect_button])

controls2_1 = widgets.Accordion(children=[text_tblnamecomp])
controls2_1.set_title(0, "Optional")
controls2 = widgets.VBox([text_tblname, column_selector, datecol_selector, metric_selector, freq_selector, date_slider, controls2_1, test_button]) # color_picker, text_xlabel, text_ylabel, int_slider

tab2 = widgets.VBox([controls2, output])

controls3 = widgets.HBox([startdate_selector, enddate_selector])

controls4 = widgets.VBox([mdl_name])

tabs = widgets.Tab([controls1, tab2, controls3, controls4])

tabs.set_title(0, "Login")
tabs.set_title(1, "Feature Details")
tabs.set_title(2, "Data Drift")
tabs.set_title(3, "Model Monitoring")
widgets.VBox([tabs])

In [19]:
plt.close('all')

In [25]:
from scipy.stats import wasserstein_distance

In [187]:
base = [-6,14,13,-1,15,23,-6,28,11,6,19,23,12,19,24,25,17,17,1,16,16,10,2,19,24,15,14,18,16,18,5,13,17,8,23,4,21,1,22,20,13,13,16,32,17,0,23,15,29,5,6,26,0,0,29,-2,18,21,23,17,25,7,18,30,3,22,3,16,16,24,9,13,22,9,18,15,28,11,15,14,-8,27,20,18,19,2,16,24,18,28,24,-4,18,25,3,20,9,20,11,19,19,0,7,12,10,12,9,10,3,1,-1,28,21,24,9,11,11,22,13,17,18,27,17,22,23,9,13,11,21,19,21,10,-2,10,15,19,17,13,-5,26,27,12,25,2,25,20,13,18,16,4,1,-1,1,26,8,12,19,16,17,10,15,19,14,21,28,8,21,14,19,26,21,15,10,12,22,0,21,11,12,-4,7,23,20,20,14,26,-1,14,24,17,6,21,8,23,21,11,18,26,11,21,13,6,17,26,15,12,19,0,-3,16,0,29,12,19,18,27,23,11,21,27,20,-6,21,8,26,27,7,22,4,16,-7,-7,-7,26,20,15,-5,28,11,6,28,20,27,30,21,9,22,30,-3,17,11,20,16,18,16,11,6,0,8,11,20,26,14,33,12,26,25,22,25,-5,24,21,31,14,23,19,12,4,23,16,19,22,3,25,-5,29,5,18,24,30,24,34,0,20,6,27,14,23,4,28,24,28,17,1,30,18,20,16,8,20,1,20,13,-5,19,12,12,2,11,17,20,21,26,25,28,3,25,20,7,29,19,26,23,18,8,15,27,10,7,14,14,24,16,20,15,16,-1,12,15,12,20,15,-4,4,22,20,18,0,17,26,7,27,-8,20,15,11,-3,2,20,19,18,9,1,21,18,0,26,8,12,20,18,10,-14,24,21,11,27,7,25,20,22,15,22,20,24,9,25,16,16,8,22,7,17,28,23,15,13,32,-1,20,20,12,26,20,13,20,18,20,10,28,4,13,23,0,25,13,13,20,14,14,-13,3,17,10,19,20,17,24,12,22,13,13,0,18,12,-5,1,11,27,0,9,8,3,21,13,6,28,9,9,17,17,14,14,15,9,12,22,34,15,7,20,19,27,28,25,-5,14,26,4,26,19,14,26,10,13,11,18,13,29,15,23,1,15,19,20,19,10,7,16,11,13,1,22,18,23,14,19,-20,7,6,23,23,16,16,-6,15,4,17,7,13,25,23,7,12,25,13,11,27,-2,9,14,21,17,32,22,15,30,13,12,8,26,27,18,13,27,23,12,26,-1,25,21,20,29,30,11,23,17,15,10,0,19,9,20,16,9,-15,9,11,10,2,-2,-1,12,3,20,20,0,26,12,16,14,18,25,18,2,17,29,6,7,19,0,17,18,14,10,21,29,10,3,26,21,22,21,9,20,8,23,14,1,0,27,16,12,26,12,23,26,1,15,20,17,9,13,12,24,0,12,2,25,28,27,4,11,8,24,0,28,-17,12,5,0,2,27,12,13,20,19,15,11,28,17,26,-1,18,0,5,26,13,4,-1,18,-24,20,19,9,13,-13,9,21,3,1,28,11,8,3,13,0,30,7,17,25,28,1,-6,13,11,0,26,14,20,11,1,2,27,28,26,26,15,26,6,19,17,15,22,25,16,18,12,6,11,24,23,8,13,24,26,19,23,8,15,23,19,22,26,5,10,11,22,16,32,12,24,12,20,-1,0,6,18,23,14,24,-21,25,20,-1,8,-1,2,30,32,11,17,9,15,12,25,16,22,0,14,6,21,29,21,28,25,15,16,1,7,16,21,12,18,16,19,27,10,12,20,30,28,11,15,20,14,27,13,3,0,22,11,10,4,10,12,10,19,20,18,19,17,15,25,14,13,27,14,24,22,24,-6,27,26,-2,16,22,7,24,14,17,17,29,7,18,25,12,28,25,12,12,19,20,8,20,24,15,13,12,25,24,23,0,12,8,20,14,-10,12,-2,22,5,14,15,16,19,26,4,4,-10,16,16,9,28,16,27,15,13,-4,12,12,19,2,0,1,12,13,20,12,23,11,7,35,27,4,25,13,15,13,4,11,30,1,22,15,4,5,11,16,0,5,29,23,27,26,6,23,10,28,2,12,10,31,16,15,18,21,18,27,26,17,12,30,19,20,10,14,14,16,17,25,13,13,2,24,6,26,18,26,27,25,12,-7,34,6,-1,26,11,24,6,10,24,28,17,14,25,32,21,16,24,11,13,17,11,17,18,27,21,19,19,8,18,29,27,13,20,13,28,24,16,1,20,10,0,25,13,26,1]

In [188]:
target = [-6,-6,11,19,17,16,10,2,15,18,18,4,20,13,16,5,0,0,21,23,17,7,30,3,16,16,13,22,18,11,15,14,-8,20,-4,18,3,20,11,19,7,12,10,28,21,24,11,18,11,19,10,19,17,-5,26,12,20,13,18,4,-1,1,19,10,14,21,8,19,15,0,12,-4,7,20,20,17,6,21,11,21,6,12,-3,12,18,27,-6,16,-7,-7,-7,20,-5,11,6,21,-3,20,18,11,12,25,22,25,14,23,4,19,25,-5,5,27,14,4,30,8,20,13,-5,19,12,12,2,11,21,29,27,10,7,14,22,7,-8,-3,2,18,21,18,20,-14,24,21,27,20,15,22,20,24,16,15,20,20,20,20,10,13,23,13,14,-13,3,20,9,6,28,17,22,7,20,27,28,-5,4,14,13,11,13,15,19,20,11,13,1,23,14,-20,7,23,-6,4,13,27,-2,30,8,26,-1,25,21,20,29,11,10,0,20,-15,9,11,12,16,18,2,6,18,14,21,10,26,20,14,0,16,1,20,13,12,28,4,24,-17,12,2,12,13,15,11,28,18,5,-1,-24,19,13,-13,9,3,11,3,13,7,17,-6,13,11,14,20,1,26,6,19,17,22,16,24,23,8,24,23,19,22,10,11,16,32,12,-1,23,14,-21,20,-1,8,2,11,9,25,16,22,0,21,29,25,15,16,7,12,16,19,10,12,20,15,27,0,10,19,20,19,15,25,14,13,24,22,24,-6,22,7,24,29,7,18,20,20,24,13,8,20,14,-10,-2,22,15,16,19,4,-10,9,27,-4,19,13,11,13,5,11,0,29,23,6,12,10,15,19,20,14,16,25,6,18,26,27,12,-7,6,24,28,17,14,13,17,11,13,1,10,1]

In [189]:
wasserstein_distance(base, target)

2.0622038935308815

In [265]:
base = [-6,14,13,-1,15,23,-6,28,11,6,19,23,12,19,24,25,17,17,1,16,16,10,2,19,24,15,14,18,16,18,5,13,17,8,23,4,21,1,22,20,13,13,16,32,17,0,23,15,29,5,6,26,0,0,29,-2,18,21,23,17,25,7,18,30,3,22,3,16,16,24,9,13,22,9,18,15,28,11,15,14,-8,27,20,18,19,2,16,24,18,28,24,-4,18,25,3,20,9,20,11,19,19,0,7,12,10,12,9,10,3,1,-1,28,21,24,9,11,11,22,13,17,18,27,17,22,23,9,13,11,21,19,21,10,-2,10,15,19,17,13,-5,26,27,12,25,2,25,20,13,18,16,4,1,-1,1,26,8,12,19,16,17,10,15,19,14,21,28,8,21,14,19,26,21,15,10,12,22,0,21,11,12,-4,7,23,20,20,14,26,-1,14,24,17,6,21,8,23,21,11,18,26,11,21,13,6,17,26,15,12,19,0,-3,16,0,29,12,19,18,27,23,11,21,27,20,-6,21,8,26,27,7,22,4,16,-7,-7,-7,26,20,15,-5,28,11,6,28,20,27,30,21,9,22,30,-3,17,11,20,16,18,16,11,6,0,8,11,20,26,14,33,12,26,25,22,25,-5,24,21,31,14,23,19,12,4,23,16,19,22,3,25,-5,29,5,18,24,30,24,34,0,20,6,27,14,23,4,28,24,28,17,1,30,18,20,16,8,20,1,20,13,-5,19,12,12,2,11,17,20,21,26,25,28,3,25,20,7,29,19,26,23,18,8,15,27,10,7,14,14,24,16,20,15,16,-1,12,15,12,20,15,-4,4,22,20,18,0,17,26,7,27,-8,20,15,11,-3,2,20,19,18,9,1,21,18,0,26,8,12,20,18,10,-14,24,21,11,27,7,25,20,22,15,22,20,24,9,25,16,16,8,22,7,17,28,23,15,13,32,-1,20,20,12,26,20,13,20,18,20,10,28,4,13,23,0,25,13,13,20,14,14,-13,3,17,10,19,20,17,24,12,22,13,13,0,18,12,-5,1,11,27,0,9,8,3,21,13,6,28,9,9,17,17,14,14,15,9,12,22,34,15,7,20,19,27,28,25,-5,14,26,4,26,19,14,26,10,13,11,18,13,29,15,23,1,15,19,20,19,10,7,16,11,13,1,22,18,23,14,19,-20,7,6,23,23,16,16,-6,15,4,17,7,13,25,23,7,12,25,13,11,27,-2,9,14,21,17,32,22,15,30,13,12,8,26,27,18,13,27,23,12,26,-1,25,21,20,29,30,11,23,17,15,10,0,19,9,20,16,9,-15,9,11,10,2,-2,-1,12,3,20,20,0,26,12,16,14,18,25,18,2,17,29,6,7,19,0,17,18,14,10,21,29,10,3,26,21,22,21,9,20,8,23,14,1,0,27,16,12,26,12,23,26,1,15,20,17,9,13,12,24,0,12,2,25,28,27,4,11,8,24,0,28,-17,12,5,0,2,27,12,13,20,19,15,11,28,17,26,-1,18,0,5,26,13,4,-1,18,-24,20,19,9,13,-13,9,21,3,1,28,11,8,3,13,0,30,7,17,25,28,1,-6,13,11,0,26,14,20,11,1,2,27,28,26,26,15,26,6,19,17,15,22,25,16,18,12,6,11,24,23,8,13,24,26,19,23,8,15,23,19,22,26,5,10,11,22,16,32,12,24,12,20,-1,0,6,18,23,14,24,-21,25,20,-1,8,-1,2,30,32,11,17,9,15,12,25,16,22,0,14,6,21,29,21,28,25,15,16,1,7,16,21,12,18,16,19,27,10,12,20,30,28,11,15,20,14,27,13,3,0,22,11,10,4,10,12,10,19,20,18,19,17,15,25,14,13,27,14,24,22,24,-6,27,26,-2,16,22,7,24,14,17,17,29,7,18,25,12,28,25,12,12,19,20,8,20,24,15,13,12,25,24,23,0,12,8,20,14,-10,12,-2,22,5,14,15,16,19,26,4,4,-10,16,16,9,28,16,27,15,13,-4,12,12,19,2,0,1,12,13,20,12,23,11,7,35,27,4,25,13,15,13,4,11,30,1,22,15,4,5,11,16,0,5,29,23,27,26,6,23,10,28,2,12,10,31,16,15,18,21,18,27,26,17,12,30,19,20,10,14,14,16,17,25,13,13,2,24,6,26,18,26,27,25,12,-7,34,6,-1,26,11,24,6,10,24,28,17,14,25,32,21,16,24,11,13,17,11,17,18,27,21,19,19,8,18,29,27,13,20,13,28,24,16,1,20,10,0,25,13,26,1]
target = [-6,-6,11,19,17,16,10,2,15,18,18,4,20,13,16,5,0,0,21,23,17,7,30,3,16,16,13,22,18,11,15,14,-8,20,-4,18,3,20,11,19,7,12,10,28,21,24,11,18,11,19,10,19,17,-5,26,12,20,13,18,4,-1,1,19,10,14,21,8,19,15,0,12,-4,7,20,20,17,6,21,11,21,6,12,-3,12,18,27,-6,16,-7,-7,-7,20,-5,11,6,21,-3,20,18,11,12,25,22,25,14,23,4,19,25,-5,5,27,14,4,30,8,20,13,-5,19,12,12,2,11,21,29,27,10,7,14,22,7,-8,-3,2,18,21,18,20,-14,24,21,27,20,15,22,20,24,16,15,20,20,20,20,10,13,23,13,14,-13,3,20,9,6,28,17,22,7,20,27,28,-5,4,14,13,11,13,15,19,20,11,13,1,23,14,-20,7,23,-6,4,13,27,-2,30,8,26,-1,25,21,20,29,11,10,0,20,-15,9,11,12,16,18,2,6,18,14,21,10,26,20,14,0,16,1,20,13,12,28,4,24,-17,12,2,12,13,15,11,28,18,5,-1,-24,19,13,-13,9,3,11,3,13,7,17,-6,13,11,14,20,1,26,6,19,17,22,16,24,23,8,24,23,19,22,10,11,16,32,12,-1,23,14,-21,20,-1,8,2,11,9,25,16,22,0,21,29,25,15,16,7,12,16,19,10,12,20,15,27,0,10,19,20,19,15,25,14,13,24,22,24,-6,22,7,24,29,7,18,20,20,24,13,8,20,14,-10,-2,22,15,16,19,4,-10,9,27,-4,19,13,11,13,5,11,0,29,23,6,12,10,15,19,20,14,16,25,6,18,26,27,12,-7,6,24,28,17,14,13,17,11,13,1,10,1]

In [186]:
wasserstein_distance(base, target)

2.0622038935308815

In [211]:
base = [-94,16,-121,-106,-84,3,120,-94,-81,40,-121,120,120,3,-121,-79,-80,-121,-80,-94,-79,16,-121,-106,16,-80,-79,147,-80,21,120,40,-121,21,147,-84,40,120,-94,-80,-121,16,-84,-121,-84,16,-94,120,-121,-81,40,40,120,40,40,113,-94,-80,3,16,-84,-111,-84,-79,16,113,-106,120,40,3,-111,40,-111,-79,113,16,120,3,-80,-79,-79,16,-94,113,-121,3,-79,147,21,-80,3,-81,120,-94,3,-81,-106,16,-121,3,21,21,113,-106,16,147,-121,21,147,147,-106,-94,-106,113,-80,113,-121,-106,-121,-121,147,-79,147,-81,21,-80,120,147,40,16,-79,-121,3,16,-94,147,-121,21,-121,-111,-94,-81,120,-121,113,147,120,-84,147,120,-111,40,-106,147,-94,120,147,-121,120,16,21,-121,-106,-79,21,-84,113,147,21,3,120,113,3,147,21,-121,120,-94,16,147,3,-106,-111,-84,113,113,147,-81,-106,-121,113,21,16,-84,16,3,40,40,-80,113,-79,120,-121,-79,-79,120,-121,-79,-79,-94,-106,21,147,120,16,-79,-121,120,3,40,21,-81,21,-94,-121,147,-80,113,-111,-84,21,-84,-94,-94,-94,-81,113,16,-94,-80,-79,-106,113,3,-81,113,-84,-111,120,120,-94,21,16,-84,147,120,-79,-79,-81,40,147,-106,-106,-111,21,-111,3,113,-121,-121,-121,16,-81,-94,120,21,113,-121,113,3,147,147,120,3,113,120,-94,113,-94,113,-111,-84,-81,-121,120,113,-94,16,147,120,-121,120,40,120,120,-121,-79,40,113,21,120,-84,40,21,40,21,-79,-94,-79,16,-121,21,-111,16,-121,-121,-80,113,-84,40,-80,3,-111,-81,-121,-81,-121,-79,147,16,-81,21,-106,147,16,113,16,-80,40,-84,-106,3,16,21,-121,21,-94,40,-84,16,-79,-94,21,-80,-111,-94,-84,-94,-80,3,147,-94,40,16,21,147,40,-94,-121,21,-94,-81,147,16,-121,-79,40,-94,-121,-80,-121,-81,-111,113,-121,-111,3,16,-80,120,16,113,-111,147,147,120,147,16,-81,-111,120,147,120,-94,120,-121,-106,113,-121,-94,3,-84,-121,-111,-79,-81,-106,16,-81,-94,-81,21,21,-79,3,-111,-94,40,16,16,3,-80,3,-81,40,-80,147,3,-94,120,147,-94,-94,40,-81,-106,-111,147,40,21,147,40,3,16,147,21,-80,3,3,-79,40,-94,-121,16,113,-121,147,3,-79,-81,-81,-80,-81,-94,16,-80,21,40,-80,-121,113,-121,-79,-79,-121,40,-80,3,120,-94,21,-111,113,16,147,147,-111,-121,147,21,-111,21,-81,21,-121,-94,-79,40,113,-80,-80,21,-94,-84,40,-79,16,-79,113,-111,40,3,-84,-79,-111,3,-94,-106,21,3,21,113,-121,-121,113,147,16,-79,-80,120,-121,3,40,3,-106,-111,-94,-81,120,-80,-121,113,21,3,40,-106,-121,-106,3,147,-80,16,-121,-94,3,-79,40,-94,-94,40,16,-106,3,-121,-94,-111,3,-79,-106,147,-79,-79,-121,40,-79,120,16,-106,3,-94,-111,120,120,40,-80,113,-121,-106,-81,40,16,21,147,-79,-121,-84,-111,-94,-106,-81,3,21,-111,21,21,40,-94,-121,113,3,16,120,-79,120,-106,3,-106,16,120,113,21,147,-111,113,-106,113,-94,-79,21,147,-94,-106,-81,21,16,3,-111,16,21,3,-121,120,-106,147,-94,16,-80,147,40,40,-121,-94,120,120,-121,21,-94,147,120,-106,-94,113,16,-111,40,-121,-94,-84,16,3,-79,113,-94,-106,147,147,-94,113,3,-84,21,40,-94,-81,-84,40,-80,-111,-81,-111,113,-121,16,120,-84,3,-79,-106,147,3,113,21,40,21,113,-84,-121,16,-121,-121,-81,-121,3,-84,-79,147,-79,-79,-121,3,-121,3,120,16,3,-106,-94,40,-111,16,-121,-80,-94,113,-81,-121,40,40,-106,40,-81,-84,21,21,3,16,-121,-81,3,3,40,21,40,3,3,120,120,113,16,-79,-106,147,16,-84,16,120,16,-80,-81,147,-106,120,-81,113,-121,-121,-84,-121,-121,-106,40,-106,-121,16,147,40,147,16,-121,120,113,21,21,-79,16,-81,3,21,120,-106,-81,-121,113,-94,113,113,-94,16,-80,16,120,21,3,21,113,3,113,113,147,-84,120,21,-121,3,-84,40,-80,113,-106,-121,21,113,16,-80,-94,-121,-121,-121,-121,-94,16,-106,-81,147,16,-79,3,-111,-121,40,21,-94,16,-111,40,120,147,-79,21,21,-94,147,-121,-111,40,-94,-106,16,-121,3,-121,120,147,40,113,113,147,120,-79,-121,3,-94,21,-84,-94,21,21,40,40,-111,3,21,-106,-81,16,21,-80,21,-79,16,-81,-94,-111,-106,120,3,21,-79,120,-84,113,-81,-79,147,-81,120,-121,-106,3,16,21,3,-81,3,147,147,-80,21,-81,16,120,113,-80,147,-94,-84,-111,-94,-81,-121,113,40,40,16,120,-79,3,16,113,-80,16,113,-106,-121,-79,-79,21,21,-80,120,-121,-80,21,40,-80,120,-121,3,16,-79,-80,40,40,-80,-121,-94,113,-121,113,-106]
target = [-94,-84,-94,40,120,-80,-79,-121,-106,-79,-80,120,40,-121,-84,-121,40,40,40,-80,16,-84,-111,-79,113,40,3,-111,-79,113,120,-79,-79,16,-94,-121,-94,3,-106,16,21,113,16,147,-121,113,-80,113,-121,147,16,-121,16,21,-121,-94,-81,-121,-84,147,120,40,147,-94,120,-121,21,-84,147,120,147,-94,3,-106,-111,113,113,21,16,-84,-79,120,-79,-79,-106,16,-121,-81,-94,-84,-94,-94,-94,113,-94,-79,-106,-84,-94,-84,120,-79,-121,-121,16,-81,-121,113,147,113,113,-94,-111,120,-121,40,113,40,21,-79,-94,-79,16,-121,21,-111,-121,-81,-81,21,-106,147,-84,-111,-94,-94,-94,40,147,-121,21,-121,-94,-121,-80,-81,-121,3,16,-80,120,-111,120,120,-121,-121,-94,-84,-79,16,-81,21,3,-94,40,-80,-111,40,3,-80,-94,16,147,3,-81,-81,-94,21,-121,-79,-79,40,3,-111,113,-121,147,21,-81,21,-94,-79,113,-94,40,-79,3,-94,113,-79,-80,-94,-81,120,-80,-121,21,-121,-106,-80,-94,3,-79,16,-79,147,40,16,120,120,-80,-121,-81,-79,-111,-106,3,-94,113,120,-79,120,21,113,-94,21,-106,21,16,16,21,3,147,16,40,-94,120,21,-94,147,-106,16,40,-121,16,3,-106,147,147,3,-84,40,-80,-111,113,-121,120,3,113,21,40,113,-81,-121,3,-79,-79,3,-121,16,-106,16,-121,-94,-121,40,40,40,21,3,-81,3,3,40,3,3,113,16,-79,147,16,16,-80,147,-106,120,-121,-121,-106,147,120,113,21,16,-81,3,21,-81,-121,113,-94,-80,16,120,113,3,113,-84,-80,113,-121,-121,-121,-121,-94,-106,-81,-79,3,-111,40,-94,40,-79,-94,-111,-121,147,-79,40,-111,21,-81,16,21,-111,-106,21,120,-121,16,21,-81,21,16,120,113,147,-94,-111,16,120,-79,3,-121,-79,-79,-121,40,-121,-106]

In [ ]:
base_sort = [-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147]

In [212]:
u_values = np.asarray(base)
v_values = np.asarray(target)

In [213]:
u_sorter = np.argsort(u_values)
v_sorter = np.argsort(v_values)

In [214]:
all_values = np.concatenate((u_values, v_values))
all_values.sort(kind="mergesort")
all_values

array([-121, -121, -121, ...,  147,  147,  147])

In [227]:
all_values2 = np.asarray([-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-121,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-111,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-106,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-94,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-84,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-81,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-80,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,-79,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,113,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147])

In [235]:
comparison = all_values2 == all_values
comparison.all()

True

In [215]:
deltas = np.diff(all_values)
deltas

array([0, 0, 0, ..., 0, 0, 0])

In [232]:
deltas2 = np.asarray([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,73,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

In [240]:
comparison = deltas == deltas2
comparison.all()

True

In [216]:
u_cdf_indices = u_values[u_sorter].searchsorted(all_values[:-1], 'right')
v_cdf_indices = v_values[v_sorter].searchsorted(all_values[:-1], 'right')

In [217]:
u_cdf_indices

array([ 105,  105,  105, ..., 1000, 1000, 1000], dtype=int64)

In [218]:
u_cdf = u_cdf_indices / u_values.size
v_cdf = v_cdf_indices / v_values.size

In [241]:
v_cdf

array([0.12073491, 0.12073491, 0.12073491, ..., 1.        , 1.        ,
       1.        ])

In [236]:
v_cdf_inx2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.12073490813648294","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.16272965879265092","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.20734908136482941","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.30708661417322836","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.34120734908136485","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.39107611548556431","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.43307086614173229","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.51706036745406825","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.58530183727034124","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.66666666666666663","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.73753280839895008","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.80839895013123364","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.87926509186351709","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951","0.94488188976377951"]

In [244]:
v_cdf_idx = np.asarray(v_cdf_inx2, dtype=float)
v_cdf_idx

array([0.        , 0.        , 0.        , ..., 0.94488189, 0.94488189,
       0.94488189])

In [245]:
comparison = v_cdf == v_cdf_idx
comparison.all()

False

In [219]:
u_cdf

array([0.105, 0.105, 0.105, ..., 1.   , 1.   , 1.   ])

In [220]:
difference = u_cdf - v_cdf
difference.sort()
print(difference)

[-0.03806037 -0.03806037 -0.03806037 ...  0.          0.
  0.        ]


In [221]:
np.abs(difference)

array([0.03806037, 0.03806037, 0.03806037, ..., 0.        , 0.        ,
       0.        ])

In [222]:
np.abs(u_cdf - v_cdf)

array([0.01573491, 0.01573491, 0.01573491, ..., 0.        , 0.        ,
       0.        ])

In [223]:
np.multiply(np.abs(u_cdf - v_cdf), deltas)

array([0., 0., 0., ..., 0., 0., 0.])

In [224]:
np.sum(np.multiply(np.abs(u_cdf - v_cdf), deltas))

6.9569422572178485

In [205]:
base_bi =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.105,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.148,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.201,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.318,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.368,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.417,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.479,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.555,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.636,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.716,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.786,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.857,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93,0.93]

In [206]:
np.asarray(base_bi)

array([0.  , 0.  , 0.  , ..., 0.93, 0.93, 0.93])

In [191]:
wasserstein_distance(base, target)

6.9569422572178485

In [76]:
wasserstein_distance([1,2,3], [2,4, 6, 8])

3.0

In [173]:
u_values = np.asarray([1,1,3])
v_values = np.asarray([2,4,6,8])

In [174]:
v_values

array([2, 4, 6, 8])

In [175]:
u_sorter = np.argsort(u_values)
v_sorter = np.argsort(v_values)
u_sorter

array([0, 1, 2], dtype=int64)

In [176]:
all_values = np.concatenate((u_values, v_values))
all_values.sort(kind="mergesort")
all_values

array([1, 1, 2, 3, 4, 6, 8])

In [177]:
deltas = np.diff(all_values)
deltas

array([0, 1, 1, 1, 2, 2])

In [178]:
# Get the respective positions of the values of u and v among the values of
# both distributions.
u_cdf_indices = u_values[u_sorter].searchsorted(all_values[:-1], 'right')
v_cdf_indices = v_values[v_sorter].searchsorted(all_values[:-1], 'right')

In [179]:
u_values[u_sorter]

array([1, 1, 3])

In [180]:
all_values[:-1]

array([1, 1, 2, 3, 4, 6])

In [181]:
u_cdf_indices

array([2, 2, 2, 3, 3, 3], dtype=int64)

In [151]:
v_cdf_indices

array([0, 1, 1, 1, 2, 3], dtype=int64)

In [144]:
u_cdf = u_cdf_indices / u_values.size
v_cdf = v_cdf_indices / v_values.size

In [153]:
v_cdf

array([0.  , 0.25, 0.25, 0.25, 0.5 , 0.75])

In [146]:
np.sum(np.multiply(np.abs(u_cdf - v_cdf), deltas))

3.0

In [154]:
u_cdf - v_cdf

array([0.33333333, 0.41666667, 0.41666667, 0.75      , 0.5       ,
       0.25      ])

In [152]:
np.abs(u_cdf - v_cdf)

array([0.33333333, 0.41666667, 0.41666667, 0.75      , 0.5       ,
       0.25      ])

In [155]:
np.multiply(np.abs(u_cdf - v_cdf), deltas)

array([0.33333333, 0.        , 0.41666667, 0.75      , 1.        ,
       0.5       ])

In [159]:
u_values = np.asarray([1,2,3])
v_values = np.asarray([8,2,4,6])
u_values.sort()
v_values.sort()


In [160]:
all_values = np.concatenate((u_values, v_values))
all_values.sort(kind="mergesort")
all_values

array([1, 2, 2, 3, 4, 6, 8])

In [161]:
deltas = np.diff(all_values)
deltas

array([1, 0, 1, 1, 2, 2])

In [162]:
# Get the respective positions of the values of u and v among the values of
# both distributions.
u_cdf_indices = u_values.searchsorted(all_values[:-1], 'right')
v_cdf_indices = v_values.searchsorted(all_values[:-1], 'right')

In [163]:
u_cdf_indices

array([1, 2, 2, 3, 3, 3], dtype=int64)

In [164]:
u_cdf = u_cdf_indices / u_values.size
v_cdf = v_cdf_indices / v_values.size

In [165]:
np.sum(np.multiply(np.abs(u_cdf - v_cdf), deltas))

3.0

In [247]:
desctest = np.asarray([3, 3, 3, 2, 2, 1])

In [248]:
scipy_results = [1.0141211631663989,
1.266974941123548,
1.6476141732283458,
2.0622038935308815,
4.2820597738287578,
6.9569422572178485]

In [249]:
kusto_results = [1.0141211631663987,
1.2669749411235478,
1.6476141732283458,
2.062203893530882,
4.2820597738287578,
6.95694225721785]

In [255]:
for i, result in enumerate(scipy_results):
    print(result - kusto_results[i])

2.220446049250313e-16
2.220446049250313e-16
0.0
-4.440892098500626e-16
0.0
-1.7763568394002505e-15


In [251]:
np.diff(np.asarray(scipy_results), np.asarray(kusto_results))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [256]:
from scipy.spatial import distance

In [276]:
len_target = len(base)

In [274]:
target_sample=target[:len(base_sample)]

In [275]:
len(target_sample)

376

In [257]:
distance.euclidean([1, 0, 0], [0, 1, 0])

1.4142135623730951

In [258]:
distance.euclidean([1, 1, 0], [0, 1, 0])

1.0

In [259]:
import math

In [264]:
math.sqrt(math.pow(1, 2) + math.pow(1, 2))

1.4142135623730951

In [268]:
distance.euclidean(base_sample, target)

ValueError: operands could not be broadcast together with shapes (381,) (376,) 